### wps_climdex_dtr

WPS wrapper for [climdex.pcic](https://cran.r-project.org/web/packages/climdex.pcic/climdex.pcic.pdf)'s [climdex.dtr](https://cran.r-project.org/web/packages/climdex.pcic/climdex.pcic.pdf#page=6) function

**climdex.dtr** computes the mean daily diurnal temperature range.  The frequency of observationcan be either monthly or annual.

In [1]:
import os
import requests
from birdy import WPSClient
from rpy2 import robjects
from urllib.request import urlretrieve
from importlib.resources import files
from tempfile import NamedTemporaryFile

from wps_tools.R import rda_to_vector, construct_r_out, test_rda_output
from wps_tools.testing import get_target_url

In [2]:
# Ensure we are in the working directory with access to the data
while os.path.basename(os.getcwd()) != "quail":
    os.chdir('../')

In [3]:
# NBVAL_IGNORE_OUTPUT
url = get_target_url("quail")
print(f"Using quail on {url}")

Using quail on https://marble-dev01.pcic.uvic.ca/twitcher/ows/proxy/quail/wps


In [4]:
quail = WPSClient(url)

#### Help for individual processes can be diplayed using the ? command (ex/ bird.process?)

In [5]:
# NBVAL_IGNORE_OUTPUT
quail.climdex_dtr?

Signature:
quail.climdex_dtr(
    climdex_input=None,
    loglevel='INFO',
    output_file='output.rda',
    freq='monthly',
    output_formats=None,
)
Docstring:
Computes the mean daily diurnal temperature range.

Parameters
----------
climdex_input : ComplexData:mimetype:`application/x-gzip`
    RDS or Rdata (.rds, .rda, .rdata) file containing R Object of type climdexInput
output_file : string
    Filename to store the output Rdata (extension .rda)
freq : {'monthly', 'annual'}string
    Time frequency to aggregate to
loglevel : {'CRITICAL', 'ERROR', 'WARNING', 'INFO', 'DEBUG', 'NOTSET'}string
    Logging level

Returns
-------
rda_output : ComplexData:mimetype:`application/x-gzip`
    Rda file containing R output data
File:      ~/github/quail/</tmp/quail-venv/lib/python3.8/site-packages/birdy/client/base.py-9>
Type:      method


#### Run wps_climdex_dtr Process with rda input

In [6]:
with NamedTemporaryFile(suffix=".rda", prefix="dtr_", dir="/tmp", delete=True) as output_file:
    output = quail.climdex_dtr(
            climdex_input=(files("tests") / "data/climdexInput.rda").resolve(),
            freq="annual",
            output_file=output_file.name,
        )
dtr_url = output.get()[0]

You can also pass rds input

In [7]:
with NamedTemporaryFile(suffix=".rda", prefix="dtr_", dir="/tmp", delete=True) as output_file:
    output = quail.climdex_dtr(
            climdex_input=(files("tests") / "data/climdexInput.rds").resolve(),
            output_file=output_file.name,
        )
dtr_url_rds = output.get()[0]

Run with multiple inputs

In [8]:
climdex_inputs = [
    (files("tests") / "data/climdexInput.rds").resolve(),
    (files("tests") / "data/climdexInput.rda").resolve(),
    (files("tests") / "data/climdex_input_multiple.rda").resolve(),
]
with NamedTemporaryFile(suffix=".rda", prefix="dtr_", dir="/tmp", delete=True) as output_file:
    output = quail.climdex_dtr(
            climdex_input=climdex_inputs,
            output_file=output_file.name,
        )

In [9]:
# NBVAL_IGNORE_OUTPUT
construct_r_out(output.get())

[[R object with classes: ('numeric',) mapped to:
  [     nan,      nan,      nan,      nan, ..., 7.883333, 7.048387,      nan,      nan],
  R object with classes: ('numeric',) mapped to:
  [     nan,      nan,      nan,      nan, ..., 7.883333, 7.048387,      nan,      nan],
  R object with classes: ('numeric',) mapped to:
  [     nan,      nan,      nan,      nan, ..., 7.883333, 7.048387,      nan,      nan],
  R object with classes: ('numeric',) mapped to:
  [     nan,      nan,      nan,      nan, ..., 7.883333, 7.048387,      nan,      nan]]]

Access the output with **rda_to_vector** or **construct_r_out** from **wps_tools.R**

In [10]:
dtr = rda_to_vector(dtr_url, "dtr1_ci")
# use print() to see whole vector
print(dtr)

    1959     1960     1961     1962     1963     1964     1965     1966 
      NA 6.870492       NA       NA 7.286575       NA 6.472603 6.230137 
    1967     1968     1969     1970     1971     1972     1973     1974 
6.578082 7.060109 6.800549 6.810989 7.092329 7.986612 7.094795 6.835890 
    1975     1976     1977     1978     1979     1980     1981     1982 
7.049589       NA       NA 6.324658 7.215068 7.183060 7.382192 8.236915 
    1983     1984     1985     1986     1987     1988     1989     1990 
8.259615       NA       NA 6.004110       NA 5.945355 6.176712 6.322802 
    1991     1992     1993     1994     1995     1996     1997     1998 
      NA 6.551913       NA       NA 6.978022 6.223140 6.282192 6.527473 
    1999     2000     2001     2002     2003     2004 
6.589286 6.407104 6.331956       NA 7.064560       NA 



In [11]:
# NBVAL_IGNORE_OUTPUT
construct_r_out(output.get())

[[R object with classes: ('numeric',) mapped to:
  [     nan,      nan,      nan,      nan, ..., 7.883333, 7.048387,      nan,      nan],
  R object with classes: ('numeric',) mapped to:
  [     nan,      nan,      nan,      nan, ..., 7.883333, 7.048387,      nan,      nan],
  R object with classes: ('numeric',) mapped to:
  [     nan,      nan,      nan,      nan, ..., 7.883333, 7.048387,      nan,      nan],
  R object with classes: ('numeric',) mapped to:
  [     nan,      nan,      nan,      nan, ..., 7.883333, 7.048387,      nan,      nan]]]

#### Testing output against expected output

In [12]:
test_rda_output(
    dtr_url, "dtr1_ci", "expected_dtr.rda", "expected_dtr_annual"
    )